# An Introduction to sklearn with Quinlan's dataset
Author: Pierre Nugues

In [1]:
from sklearn.feature_extraction import DictVectorizer
from sklearn import linear_model
from sklearn import svm
from sklearn import tree
import csv

### The dataset in a matrix format

$$
{X} =
\begin{bmatrix}
Sunny& Hot& High& False\\
%\hline
Sunny& Hot& High& True\\
%\hline
Overcast& Hot& High& False\\
%\hline
Rain& Mild& High& False\\
%\hline
Rain& Cool& Normal& False\\
Rain& Cool& Normal& True\\
%\hline
Overcast& Cool& Normal& True \\
%\hline
Sunny& Mild& High& False\\
%\hline
Sunny& Cool& Normal& False\\
%\hline
Rain& Mild& Normal& False \\
%\hline
Sunny& Mild& Normal& True\\
%\hline
Overcast& Mild& High& True \\
%\hline
Overcast& Hot& Normal& False \\
%\hline
 Rain& Mild& High& True 
\end{bmatrix}
; \mathbf{y} =
\begin{bmatrix}
    N\\
    N  \\
    P \\
    P \\
    P\\
    N\\
    P\\
 N   \\
 P   \\
P   \\
 P   \\
 P   \\
 P  \\
N   
\end{bmatrix}
$$

### Reading the dataset
We use the csv module and `DictReader()`

In [2]:
column_names = ['outlook', 'temperature', 'humidity', 'windy', 'play']
dataset = list(csv.DictReader(open('weather-nominal.csv'), 
                              fieldnames=column_names))
dataset[:3]

[{'outlook': 'sunny',
  'temperature': 'hot',
  'humidity': 'high',
  'windy': 'FALSE',
  'play': 'no'},
 {'outlook': 'sunny',
  'temperature': 'hot',
  'humidity': 'high',
  'windy': 'TRUE',
  'play': 'no'},
 {'outlook': 'overcast',
  'temperature': 'hot',
  'humidity': 'high',
  'windy': 'FALSE',
  'play': 'yes'}]

### Extracting the features 

We extract the features and the classes and we store them in `X_dict` and `y`

The class name

In [3]:
class_name = 'play'

We extract the features

In [4]:
import copy

def extract_Xy(dataset, class_name):
    X_dict = copy.deepcopy(dataset)
    y_symbols = [obs.pop(class_name, None) for obs in X_dict]
    return X_dict, y_symbols

X_dict, y = extract_Xy(dataset, class_name)

In [5]:
X_dict[:3]

[{'outlook': 'sunny',
  'temperature': 'hot',
  'humidity': 'high',
  'windy': 'FALSE'},
 {'outlook': 'sunny',
  'temperature': 'hot',
  'humidity': 'high',
  'windy': 'TRUE'},
 {'outlook': 'overcast',
  'temperature': 'hot',
  'humidity': 'high',
  'windy': 'FALSE'}]

### Vectorizing the Dataset

#### The Features

We vectorize the feature matrix and carry out a one-hot encoding

In [6]:
vec = DictVectorizer(sparse=False) # Should be true
#vec = DictVectorizer() # Should be true
vec.fit(X_dict)

DictVectorizer(sparse=False)

In [7]:
vec.feature_names_

['humidity=high',
 'humidity=normal',
 'outlook=overcast',
 'outlook=rainy',
 'outlook=sunny',
 'temperature=cool',
 'temperature=hot',
 'temperature=mild',
 'windy=FALSE',
 'windy=TRUE']

In [8]:
vec.vocabulary_

{'humidity=high': 0,
 'humidity=normal': 1,
 'outlook=overcast': 2,
 'outlook=rainy': 3,
 'outlook=sunny': 4,
 'temperature=cool': 5,
 'temperature=hot': 6,
 'temperature=mild': 7,
 'windy=FALSE': 8,
 'windy=TRUE': 9}

In [9]:
X = vec.transform(X_dict)
print(X)

[[1. 0. 0. 0. 1. 0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 1. 0. 1. 0. 0. 1.]
 [1. 0. 1. 0. 0. 0. 1. 0. 1. 0.]
 [1. 0. 0. 1. 0. 0. 0. 1. 1. 0.]
 [0. 1. 0. 1. 0. 1. 0. 0. 1. 0.]
 [0. 1. 0. 1. 0. 1. 0. 0. 0. 1.]
 [0. 1. 1. 0. 0. 1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 1. 0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 1. 1. 0. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 0. 0. 0. 0. 1. 0. 1.]
 [0. 1. 1. 0. 0. 0. 1. 0. 1. 0.]
 [1. 0. 0. 1. 0. 0. 0. 1. 0. 1.]]


#### The class vector

Scikit learn can handle strings as output

In [10]:
y

['no',
 'no',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'no']

### The Classifier

#### Building the model

With a numerical dataset, we can use a linear classifier and fit a model

In [11]:
#classifier = linear_model.LogisticRegression()
# classifier = svm.SVC()
classifier = tree.DecisionTreeClassifier(criterion='entropy')
classifier


DecisionTreeClassifier(criterion='entropy')

In [12]:
classifier.fit(X, y)

DecisionTreeClassifier(criterion='entropy')

#### Predicting the classes

We have trained a classifier and we predict the classes

In [13]:
y_predicted = classifier.predict(X)
y_predicted

array(['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes',
       'yes', 'yes', 'yes', 'no'], dtype='<U3')

In [14]:
y == y_predicted

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

### Using a test set

Should we carry out predictions on a dataset, we must use `transform()` only, and not `fit_transform()` to vectorize this set.

Although this is not a good practice, here we apply the prediction on the training set:

In [15]:
X_test_dict = X_dict
X_test = vec.transform(X_test_dict)
X_test

array([[1., 0., 0., 0., 1., 0., 1., 0., 1., 0.],
       [1., 0., 0., 0., 1., 0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0., 0., 1., 0., 1., 0.],
       [1., 0., 0., 1., 0., 0., 0., 1., 1., 0.],
       [0., 1., 0., 1., 0., 1., 0., 0., 1., 0.],
       [0., 1., 0., 1., 0., 1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0., 1., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1., 0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 1., 1., 0., 0., 1., 0.],
       [0., 1., 0., 1., 0., 0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 1., 0., 0., 1., 0., 1.],
       [1., 0., 1., 0., 0., 0., 0., 1., 0., 1.],
       [0., 1., 1., 0., 0., 0., 1., 0., 1., 0.],
       [1., 0., 0., 1., 0., 0., 0., 1., 0., 1.]])

In [16]:
y_hat = classifier.predict(X_test)
y_hat

array(['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes',
       'yes', 'yes', 'yes', 'no'], dtype='<U3')

Note that sklearn outputs strings

## Evaluation

In [17]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(classifier, X, y, cv=5,
                         scoring='accuracy')
print('Score', scores.mean())


Score 0.6333333333333333


### One more word on vectorizing

sklearn transformers must be fitted only once. Here are a few cells that show why.

A test set with new values

In [18]:
new_elt = {'outlook': 'rainy',
  'temperature': 'mild',
  'humidity': 'normal',
  'windy': 'RATHER'}

In [19]:
vec.vocabulary_

{'humidity=high': 0,
 'humidity=normal': 1,
 'outlook=overcast': 2,
 'outlook=rainy': 3,
 'outlook=sunny': 4,
 'temperature=cool': 5,
 'temperature=hot': 6,
 'temperature=mild': 7,
 'windy=FALSE': 8,
 'windy=TRUE': 9}

In [20]:
vec.transform(new_elt)

array([[0., 1., 0., 1., 0., 0., 0., 1., 0., 0.]])

In [21]:
X_test_dict.append(new_elt)
X_test_dict[-3:]

[{'outlook': 'overcast',
  'temperature': 'hot',
  'humidity': 'normal',
  'windy': 'FALSE'},
 {'outlook': 'rainy',
  'temperature': 'mild',
  'humidity': 'high',
  'windy': 'TRUE'},
 {'outlook': 'rainy',
  'temperature': 'mild',
  'humidity': 'normal',
  'windy': 'RATHER'}]

In [22]:
X_test_correct = vec.transform(X_test_dict)
X_test_correct

array([[1., 0., 0., 0., 1., 0., 1., 0., 1., 0.],
       [1., 0., 0., 0., 1., 0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0., 0., 1., 0., 1., 0.],
       [1., 0., 0., 1., 0., 0., 0., 1., 1., 0.],
       [0., 1., 0., 1., 0., 1., 0., 0., 1., 0.],
       [0., 1., 0., 1., 0., 1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0., 1., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1., 0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 1., 1., 0., 0., 1., 0.],
       [0., 1., 0., 1., 0., 0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 1., 0., 0., 1., 0., 1.],
       [1., 0., 1., 0., 0., 0., 0., 1., 0., 1.],
       [0., 1., 1., 0., 0., 0., 1., 0., 1., 0.],
       [1., 0., 0., 1., 0., 0., 0., 1., 0., 1.],
       [0., 1., 0., 1., 0., 0., 0., 1., 0., 0.]])

In [23]:
X_test_correct.shape

(15, 10)

In [24]:
X_test_wrong = vec.fit_transform(X_test_dict)
X_test_wrong

array([[1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.],
       [1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0.],
       [0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0.],
       [0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0.]])

In [25]:
vec.vocabulary_

{'outlook=sunny': 4,
 'temperature=hot': 6,
 'humidity=high': 0,
 'windy=FALSE': 8,
 'windy=TRUE': 10,
 'outlook=overcast': 2,
 'outlook=rainy': 3,
 'temperature=mild': 7,
 'temperature=cool': 5,
 'humidity=normal': 1,
 'windy=RATHER': 9}

In [26]:
X_test_wrong.shape

(15, 11)